In [ ]:
!nvidia-smi

Fri Jun 10 05:06:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%pip install -q transformers[sentencepiece] datasets wandb

     |████████████████████████████████| 4.2 MB 15.3 MB/s 
     |████████████████████████████████| 346 kB 76.0 MB/s 
     |████████████████████████████████| 1.8 MB 63.1 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 75.5 MB/s 
     |████████████████████████████████| 1.1 MB 61.5 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 74.3 MB/s 
     |████████████████████████████████| 127 kB 74.9 MB/s 
     |████████████████████████████████| 145 kB 77.3 MB/s 
     |████████████████████████████████| 181 kB 70.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 271 kB 76.3 MB/s 
     |████████████████████████████████| 144 kB 73.3 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 112 kB 66.2 MB/s 
     |████████████████

### In this notebook we experiment by finetuning a bert-for-patents model on competition data by adding patent section as special token to the tokenizer vocab.

In [ ]:
%pip install bitsandbytes-cuda112

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 13.3 MB/s 


In [ ]:
# This tests if the installation was successful
!wget https://gist.githubusercontent.com/TimDettmers/1f5188c6ee6ed69d211b7fe4e381e713/raw/4d17c3d09ccdb57e9ab7eca0171f2ace6e4d2858/check_bnb_install.py && python check_bnb_install.py

--2022-06-10 05:07:16--  https://gist.githubusercontent.com/TimDettmers/1f5188c6ee6ed69d211b7fe4e381e713/raw/4d17c3d09ccdb57e9ab7eca0171f2ace6e4d2858/check_bnb_install.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 327 [text/plain]
Saving to: ‘check_bnb_install.py’

check_bnb_install.p 100%[===================>]     327  --.-KB/s    in 0s      

2022-06-10 05:07:16 (21.1 MB/s) - ‘check_bnb_install.py’ saved [327/327]

SUCCESS!
Installation was successful!


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch.multiprocessing as mp
import wandb
import warnings
import bitsandbytes as bnb
from transformers.trainer_pt_utils import get_parameter_names

warnings.filterwarnings('ignore')


In [ ]:
class WandbConfig:
    WANDB_KEY = "c5e2877bf080e6b62fcc57231c91e3a1455f97d0"
    WANDB_RUN_NAME = "deberta-v2-xlarge-run1"
    WANDB_PROJECT = "huggingface"
    USE_WANDB = True  

class TrainingArgs:
    weight_decay = 0.01
    learning_rate = 7e-6
    warmup_ratio = 0.1
    gradient_accumulation_steps = 8
    fp16 = True
    lr_scheduler_type = "linear"
    # Number of checkpoints to save for each model
    save_total_limit = 1
    gradient_checkpointing = False
    max_steps=-1 # set >0 to limit
    report_to="wandb"
    max_grad_norm=50.0

class Config:  
    MODEL_NAME = "deberta-v2-xlarge"
    DATA_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/data/"
    VAL_PREDS_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/preds/"
    # location where trained model weights are saved
    OUT_DIR = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v2-xlarge/"
    RUNTIME = "COLAB"
    RANDOM_STATE = 42
    BATCH_SIZE = 8
    NUM_LABELS = 1
    NUM_FOLDS = 5
    RUN_ALL_FOLDS = True
    NUM_EPOCHS = 4
    NUM_WORKERS = mp.cpu_count()
    TRANSFORMER_CHECKPOINT = "microsoft/deberta-v2-xlarge"
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SUBSET_ROWS_FRAC = 0.05
    TRAIN_ON_SUBSET = False
    RANDOM_SEED = 42
    ADAM_BITS = 8

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
def empty_gdrive_trash():
    deleted_file_name = []
    for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
        file_name = a_file['title']
        deleted_file_name.append(file_name)
        # delete the file permanently.
        a_file.Delete()
    print("The below files were cleared from trash")
    print(deleted_file_name)

In [ ]:
if WandbConfig.USE_WANDB:
    if Config.RUNTIME == "KAGGLE":
        pass
        #wandb_login()
    else:
        wandb.login(key=WandbConfig.WANDB_KEY)        

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
empty_gdrive_trash()

The below files were cleared from trash
[]


In [ ]:
df_train = pd.read_csv(Config.DATA_PATH + "train.csv")
df_test = pd.read_csv(Config.DATA_PATH + "test.csv")
df_titles = pd.read_csv(Config.DATA_PATH + "titles.csv")

In [ ]:
df_train["section"] = df_train.context.str[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder

anchor_encoder = LabelEncoder()
df_train["anchor_map"] = anchor_encoder.fit_transform(df_train["anchor"])
# Score is not really a continuous value here as there are just five distinct values. But since it is float it needs to be converted
# to categorical value before we can perform stratified split on score
df_train["score_map"] = df_train["score"].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})

In [ ]:
from sklearn import model_selection

def strat_group_kfold_dataframe(df, target_col_name, group_col_name, num_folds=Config.NUM_FOLDS):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df = df.sample(frac=1, random_state=Config.RANDOM_SEED).reset_index(drop=True)
    # get the target data
    y = df[target_col_name].values    
    groups = df[group_col_name].values
    # stratify data using anchor as group and score as target
    skf = model_selection.StratifiedGroupKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_SEED)
    for fold, (train_index, val_index) in enumerate(skf.split(X=df, y=y, groups=groups)):
        df.loc[val_index, "kfold"] = fold        
    return df     

In [ ]:
def strat_kfold_dataframe(df, target_col_name, num_folds=5):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df.sample(frac=1, random_state=Config.RANDOM_STATE).reset_index(drop=True)
    y = df[target_col_name].values
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_STATE)
    # stratification is done on the basis of y labels, a placeholder for X is sufficient
    for fold, (train_idx, val_idx) in enumerate(skf.split(X=df, y=y)):
        df.loc[val_idx, "kfold"] = fold
    return df

In [ ]:
if Config.TRAIN_ON_SUBSET:
    print(f"Selecting {Config.SUBSET_ROWS_FRAC * 100}% training data")
    df_train = df_train.sample(frac=Config.SUBSET_ROWS_FRAC, random_state=Config.RANDOM_SEED).reset_index(drop=True)

# Since the target column (score) is continuous, we need to create bins out of the target column
df_train.loc[:, "bins"] = pd.cut(df_train.score, bins=5, labels=[0,1,2,3,4])
df_train = strat_kfold_dataframe(df_train, target_col_name="bins", num_folds=Config.NUM_FOLDS)

# Now do a stratified group k fold on the bins column (which is a categorical column) and anchor as groups
# df_train = strat_group_kfold_dataframe(df_train, target_col_name="score_map", group_col_name="anchor_map", num_folds=Config.NUM_FOLDS)            
# drop the bin column
df_train = df_train.drop(["bins"], axis=1)
df_train = df_train.drop(["anchor_map", "score_map"], axis=1)

In [ ]:
# Let us check if the stratification has been done correctly
# The mean of score column should be similar across folds 
fold_score_mean = []
for fold in range(Config.NUM_FOLDS):
    fold_score_mean.append(np.mean(df_train[df_train.kfold == fold].score.values))
fold_score_mean

[0.36209732693625774,
 0.36209732693625774,
 0.36209732693625774,
 0.3619755963805868,
 0.3620441458733205]

In [ ]:
df_titles.head()

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


In [ ]:
df_train = pd.merge(
    left = df_train,
    right = df_titles[["code", "title"]],
    how = "inner",
    left_on = "context",
    right_on = "code"
)

In [ ]:
df_train

,id,anchor,target,context,score,section,kfold,code,title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,1,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,3,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,2,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,3,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,0,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
...,...,...,...,...,...,...,...,...,...
36468,718f1c6953e3942f,undulation,undulatory swimmers,B31,0.00,B,0,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA..."
36469,4dc407e6d0aa7844,undulation,voltage fluctuate,B31,0.00,B,0,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA..."
36470,de69548ad79caccc,web transfer,transfer from web,B31,0.75,B,3,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA..."
36471,6620317413e6e03f,web transfer,transfer to web,B31,0.25,B,2,B31,"MAKING ARTICLES OF PAPER, CARDBOARD OR MATERIA..."


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(Config.TRANSFORMER_CHECKPOINT)
# DataCollatorWithPadding pads each batch to the longest sequence length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained(Config.TRANSFORMER_CHECKPOINT, num_labels=Config.NUM_LABELS)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/633 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
deberta_vocab = tokenizer.get_vocab()
len(deberta_vocab)

128001

In [ ]:
df_train['sectok'] = '[' + df_train.section + ']'
sectoks = list(df_train.sectok.unique())
sep = '[s]'
sectoks.append(sep)
tokenizer.add_special_tokens({'additional_special_tokens': sectoks})

9

In [ ]:
#model.resize_token_embeddings(len(tokenizer.get_vocab()))

In [ ]:
#sep = tokenizer.sep_token

In [ ]:
df_train["inputs"] = df_train.sectok + sep + df_train.anchor + sep + df_train.target + sep + df_train.title 
df_train.head()

,id,anchor,target,context,score,section,kfold,code,title,sectok,inputs
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,1,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]abatement of pollution[s]FUR...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,3,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]act of abating[s]FURNITURE; ...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,2,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]active catalyst[s]FURNITURE;...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,3,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]eliminating process[s]FURNIT...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,0,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]forest region[s]FURNITURE; D...


In [ ]:
def tokenize_text(tokenizer, with_labels, row):
    encoding = tokenizer(
        text = row["inputs"],
        padding = False,
        truncation = True
    )
    if with_labels:
        encoding["labels"] = row["score"]
    return encoding

In [ ]:
from functools import partial

preprocess_train_data = partial(tokenize_text, tokenizer, True)  
preprocess_test_data = partial(tokenize_text, tokenizer, False)  

In [ ]:
def get_fold_dls(fold, df):
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    ds_train_raw = Dataset.from_pandas(train_df)
    ds_valid_raw = Dataset.from_pandas(valid_df)
    raw_ds_col_names = ds_train_raw.column_names    
    ds_train = ds_train_raw.map(preprocess_train_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)
    ds_valid = ds_valid_raw.map(preprocess_train_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)    
    return train_df, valid_df, ds_train, ds_valid

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [ ]:
def get_oof_preds(trainer, ds_val, df_val_fold):
    oof_outputs = trainer.predict(ds_val)
    oof_predictions = oof_outputs.predictions.reshape(-1)
    df_val_fold["val_preds"] = oof_predictions
    return df_val_fold

In [ ]:
import math
import transformers

# Thanks to Nicolas Broad. Taken from https://www.kaggle.com/code/nbroad/8-bit-adam-optimization/notebook
def get_optimizer(model, args, train_dataset):    
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

    # These are the only changes you need to make. The first part sets the optimizer to use 8-bits
    # The for loop sets embeddings to use 32-bits
    if Config.ADAM_BITS == 32:
        optimizer = bnb.optim.Adam32bit(optimizer_grouped_parameters, lr=args.learning_rate)
    if Config.ADAM_BITS == 8:
        optimizer = bnb.optim.Adam8bit(optimizer_grouped_parameters, lr=args.learning_rate)
        
    # Thank you @gregorlied https://www.kaggle.com/nbroad/8-bit-adam-optimization/comments#1661976
    for module in model.modules():
        if isinstance(module, torch.nn.Embedding):
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                module, 'weight', {'optim_bits': 32}
            )            

    num_update_steps_per_epoch = len(train_dataset) // args.per_device_train_batch_size // args.gradient_accumulation_steps
    if args.max_steps == -1 or args.max_steps is None:
        args.max_steps = Config.NUM_EPOCHS * num_update_steps_per_epoch
    else:
        num_train_epochs = args.max_steps / num_update_steps_per_epoch
        #print(f"num_train_epochs = {num_train_epochs}")
        args.num_train_epochs = math.ceil(num_train_epochs)
        
    if args.warmup_ratio is not None:
        args.num_warmup_steps = int(args.warmup_ratio * args.max_steps)

    lr_scheduler = transformers.get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=args.max_steps,
    )
    return optimizer, lr_scheduler

In [ ]:
df_train.kfold.value_counts()

1    7295
2    7295
0    7295
3    7294
4    7294
Name: kfold, dtype: int64

In [ ]:
import gc

df_val_preds = pd.DataFrame()
tok_vocab = tokenizer.get_vocab()
for fold in [4]: #range(Config.NUM_FOLDS):
    fold_str = f"fold{fold}"
    print(f"Running training for {Config.MODEL_NAME} {fold_str}")
    df_train_fold, df_val_fold, ds_train, ds_val = get_fold_dls(fold, df_train)
    training_args = TrainingArguments(
        output_dir=Config.OUT_DIR + fold_str,
        evaluation_strategy="epoch",
        save_strategy='no',        
        num_train_epochs=Config.NUM_EPOCHS,
        per_device_train_batch_size=Config.BATCH_SIZE,
        per_device_eval_batch_size=Config.BATCH_SIZE,
        warmup_ratio=TrainingArgs.warmup_ratio,
        weight_decay=TrainingArgs.weight_decay,
        learning_rate=TrainingArgs.learning_rate,    
        gradient_accumulation_steps=TrainingArgs.gradient_accumulation_steps,
        fp16=TrainingArgs.fp16,
        lr_scheduler_type=TrainingArgs.lr_scheduler_type,
        save_total_limit=TrainingArgs.save_total_limit,
        gradient_checkpointing=TrainingArgs.gradient_checkpointing,
        max_steps=TrainingArgs.max_steps,
        report_to=TrainingArgs.report_to,
        max_grad_norm=TrainingArgs.max_grad_norm
    )
    model = AutoModelForSequenceClassification.from_pretrained(Config.TRANSFORMER_CHECKPOINT, num_labels=Config.NUM_LABELS)
    optimizer, lr_scheduler = get_optimizer(model, training_args, ds_train)
    print(f"len(tokenizer_vocab) = {len(tok_vocab)}")
    model.resize_token_embeddings(len(tok_vocab))    
    trainer = Trainer(
        model=model,                         # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=ds_train,              # training dataset
        eval_dataset=ds_val,                 # evaluation dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
        data_collator=data_collator,
        tokenizer=tokenizer,
        optimizers=(optimizer, lr_scheduler)
    )
    trainer.train()
    trainer.save_model(Config.OUT_DIR + fold_str)
    df_val_fold = get_oof_preds(trainer, ds_val, df_val_fold) 
    display(df_val_fold.head())
    df_val_preds = pd.concat([df_val_preds, df_val_fold], axis=0)
    # export the oof predictions to csv for later use in stacking
    if Config.RUNTIME != "KAGGLE":
        df_val_preds.to_csv(Config.VAL_PREDS_PATH + f"df_train_oof_preds_{Config.MODEL_NAME}_{fold_str}.csv")
    else:
        df_val_preds.to_csv("/kaggle/working/df_train_oof_preds.csv")
    print(f"Saved OOF predictions for fold {fold}")    
    del model, trainer, optimizer, lr_scheduler
    gc.collect()
    torch.cuda.empty_cache()
    # Empty the trash to clear gdrive disk space
    empty_gdrive_trash()
    if not Config.RUN_ALL_FOLDS:
        break


Running training for deberta-v2-xlarge fold4


  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v2-xlarge were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.

...

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend
***** Running training *****
  Num examples = 29179
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 1820
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: bkanupam. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.024387,0.809711
2,0.051500,0.021696,0.837038
3,0.023300,0.021847,0.847235
3,0.018100,0.019658,0.853770


***** Running Evaluation *****
  Num examples = 7294
  Batch size = 8
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 8
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 8
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v2-xlarge/fold4
Configuration saved in /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v2-xlarge/fold4/config.json
Model weights saved in /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v2-xlarge/fold4/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v2-xlarge/fold4/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v2-xlarge/fold4/special_tokens_map.

,id,anchor,target,context,score,section,kfold,code,title,sectok,inputs,val_preds
0,067203128142739c,abatement,greenhouse gases,A47,0.25,A,4,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]greenhouse gases[s]FURNITURE...,0.201904
1,061d17f04be2d1cf,abatement,increased rate,A47,0.25,A,4,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]increased rate[s]FURNITURE; ...,0.050812
2,0a425937a3e86d10,abatement,minimising sounds,A47,0.50,A,4,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]minimising sounds[s]FURNITUR...,0.453857
3,b3832eac81b73dfd,abatement,pollution abatement incinerator,A47,0.50,A,4,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]abatement[s]pollution abatement incinera...,0.391602
4,4603e13580940257,adhesive mounting,adhering mount,A47,0.75,A,4,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,[A],[A][s]adhesive mounting[s]adhering mount[s]FUR...,1.034180


Saved OOF predictions for fold 4
The below files were cleared from trash
[]


In [ ]:
# Calculate the CV score
predictions = df_val_preds['val_preds'].values
labels = df_val_preds['score'].values
eval_preds = predictions, labels
cv_metric_dict = compute_metrics(eval_preds)
print(f"CV score = {cv_metric_dict}")

CV score = {'pearson': 0.8537702533689654}


In [ ]:
df_val_preds["score_pred_diff"] = df_val_preds.apply(lambda row: abs(row["val_preds"] - row["score"]), axis=1)
df_val_preds = df_val_preds.sort_values(by=["score_pred_diff"], ascending=False)
df_val_preds_diff = df_val_preds[df_val_preds.score_pred_diff > 0.2]
len(df_val_preds_diff)

1055

In [ ]:
df_val_preds_diff[["anchor", "target", "context", "score", "val_preds", "score_pred_diff"]]

,anchor,target,context,score,val_preds,score_pred_diff
7246,pictorial image,pictorial representation,G09,0.00,0.771484,0.771484
6424,close gate,pulse gate,F03,0.75,0.004025,0.745975
5723,coupling factor,quality factor,H03,0.75,0.036255,0.713745
4410,pipe box,pipe bundle,C21,0.00,0.701172,0.701172
3705,selected box,selected trunk,B28,0.75,0.050537,0.699463
...,...,...,...,...,...,...
4481,organic alkaline,amino based,C11,0.50,0.299561,0.200439
1903,transfer onto substrate,transfer to substrate,B41,1.00,0.799805,0.200195
3344,radio wave transmission,half of communication,G01,0.50,0.299805,0.200195
4121,congruency,calibration,C02,0.25,0.450195,0.200195
